In [1]:
#IMPORT LIBRARIES AND PACKAGES
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm
from random import randint
from time import sleep
import requests
import yfinance as yf




In [2]:
#PREPARATION FOR SELENIUM
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&datatype=csv&outputsize=compact&apikey=9YUPW0WRVEKUFF9L'
df_stock = pd.read_csv(url)
df_stock = df_stock.drop(columns=['volume'])
df_stock.to_csv('stock_data.csv', index=False)


In [136]:
#OPEN DRIVER
driver = webdriver.Chrome()


In [137]:
#GO TO SENATOR TRADING PAGE
url = "https://trendspider.com/markets/congress-trading/"
driver.get(url)
time.sleep(5)


In [138]:
headers = driver.find_elements(By.CSS_SELECTOR, "#main-content table thead th")
header_names = [header.text.strip() for header in headers]


In [139]:
#GET THE INFORMATION FROM THE FIRST PAGE
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text.strip().replace('\n', ' ') for col in cols])

In [140]:
#GET THE INFORMATION FROM THE FIRST 811 PAGES (ORIGINAL DATASET)
for i in tqdm(range(2, 811)):
    check = f'[aria-label="Current Page, Page {i}."]'
    site = f'[aria-label="Go to page {i}."]'
    while driver.find_elements(By.CSS_SELECTOR, check) == []:
        try:
            driver.find_elements(By.CSS_SELECTOR, site)[0].click()
        except Exception as e:
            time.sleep(randint(10,20)/10)
    

    sleep(randint(10,20)/10)
    rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")
    
    sleep(randint(10,20)/10)
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text.strip().replace('\n', ' ') for col in cols])

100%|██████████| 809/809 [1:58:56<00:00,  8.82s/it]  


In [141]:
#SPLIT COLUMN ABOUT TRANSACTIONS INTO 2 COLUMNS
df = pd.DataFrame(data, columns=header_names)
df[['Type', 'Amount']] = df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
df = df.drop(columns=['Transaction', 'Excess return *'])


In [181]:
#EXPORT AND QUITTING DRIVER
#df.to_csv('senators_trading.csv', index=False) #DISABLED DUE TO POTENTIAL MISSCLICK
driver.quit()


In [21]:

df = pd.read_csv('senators_trading.csv')

UPDATE (BOTH INFORMATION ABOUT ASSETS and ALSO SENATORS_TRADING DATASET)

In [67]:
#LOADING THE CORE DATASET
data_trading = pd.read_csv("senators_trading.csv")

In [68]:
#OPEN DRIVER
driver = webdriver.Chrome()

In [69]:
#GO TO SENATORS TRADING WEBPAGE
url = "https://trendspider.com/markets/congress-trading/"
driver.get(url)
time.sleep(5)


In [70]:
headers = driver.find_elements(By.CSS_SELECTOR, "#main-content table thead th")
header_names = [header.text.strip() for header in headers]

In [71]:
#GET THE DATA FROM THE FIRST PAGE
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text.strip().replace('\n', ' ') for col in cols])

    


df = pd.DataFrame(data, columns=header_names)
df[['Type', 'Amount']] = df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
df = df.drop(columns=['Transaction', 'Excess return *'])

#SAVE LAST ROW FROM THE TEMPORAL DATASET (WILL BE USED TO CHECK, IF ORIGINAL DATASET CONTAINS THE UPDATED DATA)
last_note = df.iloc[-1]

In [72]:
#CONTINUE TO 2nd ... PAGE. CONITINUE UNTIL THE UPDATED DATASET STARTS TO INTERSECT WITH THE ORIGINAL DATASET
i = 2
while ((data_trading == last_note).all(axis=1)).any() == False:
    data = []
    check = f'[aria-label="Current Page, Page {i}."]'
    site = f'[aria-label="Go to page {i}."]'
    while driver.find_elements(By.CSS_SELECTOR, check) == []:
        try:
            driver.find_elements(By.CSS_SELECTOR, site)[0].click()
        except Exception as e:
            time.sleep(randint(10,20)/10)
    sleep(randint(10,50)/10)
    rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")
    
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text.strip().replace('\n', ' ') for col in cols])

    help_df = pd.DataFrame(data, columns=header_names)
    help_df[['Type', 'Amount']] = help_df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
    help_df = help_df.drop(columns=['Transaction', 'Excess return *'])

    df = pd.concat([df, help_df])


    i += 1
    last_note = df.iloc[-1]

    print(last_note)

driver.quit()

#LEAVE ONLY RECORDS FROM TEMPORAL DATASET, WHICH ARE NOT IN THE ORIGINAL DATASET (CANNOT USE DROP_DUPLICATES DUE TO SIMILARITY BETWEEN SOME TRADES)
mask = ~df.apply(tuple, axis=1).isin(data_trading.apply(tuple, axis=1))
result = df[mask]

#SAVE THE ORIGINAL SYMBOLS FROM THE ORIGINAL CORE DATASET (WHICH CORRESPOND TO THE ORIGINAL DATASET WITH INFO ABOUT STOCKS,ETF,MUTUAL FUNDS...)
original_symbols = data_trading.Stock.apply(lambda x: x.split(" ")[0]).drop_duplicates()

#CONNECT TEMPORAL AND CORE DATASET TO GET UP TO DATE DATASET
data_trading = pd.concat([result, data_trading])

#ADD SYMBOLS TO DATASET
data_trading["ABR"] = data_trading.Stock.apply(lambda x: x.split(" ")[0])


In [77]:
#GET THE SYMBOLS FROM THE TEMPORAL DATASET AND LEAVE ONLY THOSE ETH,STOCKS..., FOR WHICH THE INFORMATION ARE MISSING
result["ABR"] = result.Stock.apply(lambda x: x.split(" ")[0])
result_symbols = result.ABR.drop_duplicates()

additional_symbols = ~result_symbols.isin(original_symbols)


C:\Users\Dario Mikuš\AppData\Local\Temp\ipykernel_17248\80750706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["ABR"] = result.Stock.apply(lambda x: x.split(" ")[0])


In [78]:
#ITTER THROUGH SYMBOLS WITHOUT INFORMATION TO OBTAIN MISSING INFORMATION FROM YAHOO_FINANCE
#MUST CREATE SEVERAL DATASETS DUE TO DIFFERENCES BETWEEN THE ASSET TYPES
equity = pd.DataFrame()
mutual_fund = pd.DataFrame()
etf = pd.DataFrame()
other = pd.DataFrame()

# ITERATE THROUGH SYMBOLS AND GAIN MISSING INFORMATION
for symbol in tqdm(additional_symbols):
    try:
        stock = yf.Ticker(symbol)
        time.sleep(randint(10,20)/10)

        stock_info = pd.json_normalize(stock.info)

        if 'quoteType' in stock_info.columns:
            quote_type = stock_info['quoteType'][0]
        
    
            if quote_type == 'ETF':
                etf = pd.concat([etf, stock_info], ignore_index=True)
            elif quote_type == 'EQUITY':
                equity = pd.concat([equity, stock_info], ignore_index=True)
            elif quote_type == 'MUTUALFUND':
                mutual_fund = pd.concat([mutual_fund, stock_info], ignore_index=True)
            else:
                other = pd.concat([other, stock_info], ignore_index=True)
        else:
             print(f"quoteType not found for {symbol}. Skipping.")
    except Exception as e:
        continue

100%|██████████| 1/1 [00:00<?, ?it/s]


In [79]:
#ORIGINAL DATASET
info_dataset = pd.read_csv("info_dataset.csv")

#ETF
try:
    data_etf = etf[["quoteType", "symbol", "category", "currency"]]
except:
    data_etf = pd.DataFrame(columns = ["quoteType", "symbol", "category", "currency"])

#EQUITY
try:
    data_equity = equity[["quoteType", "symbol", "country", "industry", "sector", "financialCurrency"]]

    data_equity = data_equity.rename(columns = {"industry" : "category", "financialCurrency" : "currency"})
except:
    data_equity = pd.DataFrame(columns = ["quoteType", "symbol", "country", "industry", "sector", "financialCurrency"])


#MUTUAL FUND
try:
    data_mutual_fund = mutual_fund[["quoteType", "symbol", "currency"]]
except:
    data_mutual_fund = pd.DataFrame(columns = ["quoteType", "symbol", "currency"])

#OTHER
try:
    data_other = other[["quoteType", "symbol", "currency"]]
except:
    data_other = pd.DataFrame(columns = ["quoteType", "symbol", "currency"])

#MERGE ALL DATASETS TO GET UP TO DATE DATASET WITH INFORMATION ABOUT ASSETS
info_dataset = pd.concat([info_dataset, data_etf, data_equity, data_mutual_fund, data_other], ignore_index=True, join='outer')

In [4]:
pd.options.display.max_columns = None


GETTING INFO ABOUT STOCKS_ETF_FUNDS... (CORE DATASET)

In [33]:
#GET THE SYMBOLS, TO BE USED IN YAHOO_FINANCE API
data_trading["ABR"] = data_trading.Stock.apply(lambda x: x.split(" ")[0])

In [36]:
#GAIN UNIQUE SYMBOLS
symbols = data_trading.ABR.drop_duplicates()

In [50]:
#CREATE EMPTY DATASETS DUE TO DIFFERENCES BETWEEN INFORMATION PROVIDED FOR DIFFERENT ASSETS
equity = pd.DataFrame()
mutual_fund = pd.DataFrame()
etf = pd.DataFrame()
other = pd.DataFrame()

In [ ]:
# ITERATE THROUGH ALL SYMBOLS
for symbol in tqdm(symbols):
    try:
        stock = yf.Ticker(symbol)
        time.sleep(randint(10,20)/10)

        stock_info = pd.json_normalize(stock.info)

        if 'quoteType' in stock_info.columns:
            quote_type = stock_info['quoteType'][0]
        
    
            if quote_type == 'ETF':
                etf = pd.concat([etf, stock_info], ignore_index=True)
            elif quote_type == 'EQUITY':
                equity = pd.concat([equity, stock_info], ignore_index=True)
            elif quote_type == 'MUTUALFUND':
                mutual_fund = pd.concat([mutual_fund, stock_info], ignore_index=True)
            else:
                other = pd.concat([other, stock_info], ignore_index=True)
        else:
             print(f"quoteType not found for {symbol}. Skipping.")
    except Exception as e:
        continue

In [66]:
#EXPORT (NO LONGER USED)
#other.to_csv("other.csv")
#etf.to_csv("etf.csv")
#equity.to_csv("equity.csv")
#mutual_fund.to_csv("mutual_fund.csv")

In [2]:
#LOAD (NO LONGER USED DUE TO CREATION OF OVERAL DATASET)
other = pd.read_csv("other.csv")
equity = pd.read_csv("equity.csv")
mutual_fund = pd.read_csv("mutual_fund.csv")
etf = pd.read_csv("etf.csv")

In [14]:
#INFORMATIVE DATA FOR ETF
data_etf = etf[["quoteType", "symbol", "category", "currency"]]

In [17]:
#INFORMATIVE DATA FOR EQUITY
data_equity = equity[["quoteType", "symbol", "country", "industry", "sector", "financialCurrency"]]

data_equity = data_equity.rename(columns = {"industry" : "category", "financialCurrency" : "currency"})

In [ ]:
#INFORMATIVE DATA FOR MUTUAL FUNDS
data_mutual_fund = mutual_fund[["quoteType", "symbol", "currency"]]

In [ ]:
#INFORMATIVE DATA FOR OTHER TYPES (INDEX, NONE)
data_other = other[["quoteType", "symbol", "currency"]]

In [20]:
#CREATE COMPLET INFO DATASET
info_dataset = pd.DataFrame(columns= ["symbol", "quoteType", "country", "category", "sector", "currency", "category"])

info_dataset = pd.concat([data_etf, data_equity, data_mutual_fund, data_other], ignore_index=True, join='outer')

In [ ]:
#EXPORT
info_dataset.to_csv("info_dataset.csv")